In [1]:
%matplotlib notebook
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
from cg2d_total_mirror import calc_V, sweep

In [2]:
@njit
def calcArrs(Nsweeps,Tvec,Lvec,J = 0):
    '''Performs metropolis sweeps for different parameters and returns simulation data'''
    N_L = Lvec.shape[0]
    N_T = Tvec.shape[0]
    Harr = np.zeros((N_L,N_T,Nsweeps))
    Narr = np.zeros((N_L,N_T,Nsweeps))
    qy_flow_arr = np.zeros((N_L,N_T,Nsweeps))
    
    for i,L in enumerate(Lvec):
        #print('L = '+str(L)+' is beginning')
        lambd = 4*L
        V = calc_V(L,2*L,lambd)
        for j,T in enumerate(Tvec):
            #print('T = '+str(T)+' is beginning')
            lattice = np.zeros((L,2*L))
            H = 0
            N = 0
            qy_flow = 0
            for k in range(Nsweeps):
                lattice, H, N, qy_flow = sweep(J,lattice,V,T,H,N,qy_flow)
                Harr[i,j,k] = H
                Narr[i,j,k] = N
                qy_flow_arr[i,j,k] = qy_flow
        #print('L = '+str(L)+' is done')
    
    return Harr, Narr, qy_flow_arr
@njit
def calcArrsCurrent(Nsweeps,Tvec,Lvec,Jvec):
    '''Performs metropolis sweeps for different parameters and returns simulation data'''
    N_J = Jvec.shape[0]
    N_L = Lvec.shape[0]
    N_T = Tvec.shape[0]
    Harr = np.zeros((N_J,N_L,N_T,Nsweeps))
    Narr = np.zeros((N_J,N_L,N_T,Nsweeps))
    qy_flow_arr = np.zeros((N_J,N_L,N_T,Nsweeps))
    
    for i,J in enumerate(Jvec):
        for j,L in enumerate(Lvec):
            #print('L = '+str(L)+' is beginning')
            lambd = 4*L
            V = calc_V(2*L,2*L,lambd)
            for k,T in enumerate(Tvec):
                #print('T = '+str(T)+' is beginning')
                lattice = np.zeros((2*L,2*L))
                H = 0
                N = 0
                qy_flow = 0
                for l in range(Nsweeps):
                    lattice, H, N, qy_flow = sweep(J,lattice,V,T,H,N,qy_flow)
                    Harr[i,j,k,l] = H
                    Narr[i,j,k,l] = N
                    qy_flow_arr[i,j,k,l] = qy_flow
            #print('L = '+str(L)+' is done')
    
    return Harr, Narr, qy_flow_arr

In [8]:
Nsweeps = 100000
Tvec = np.linspace(0.10,0.30,11)
Lvec = np.array([4])
Jvec = 0.1*np.logspace(0, 4, 20, base =np.e)
Harr, Narr, qy_flow_arr = calcArrsCurrent(Nsweeps,Tvec,Lvec,Jvec)
#timeit calcArrsCurrent(Nsweeps,Tvec,Lvec,Jvec)

In [9]:
np.save('cgdata_17feb.npy', np.array([Harr, Narr, qy_flow_arr]))

In [10]:
gen_data = np.array(["Nsweeps = 10000", "Tvec = np.linspace(0.10,0.30,11)", "Lvec = np.array([4])", "0.1*np.logspace(0, 4, 20, base =np.e)"])
np.savetxt('cgdata_17feb.txt', gen_data, fmt="%s")